# Creating a Discussion

The simplest task you can accomplish with this library is to create a small discussion between LLMs. 

This guide will teach you the basic setup of the library. You will understand how to setup models, user-agents and how to coordinate them in a discussion. By the end of htis guide, you will be able to run a small discussion with a moderator and save it to the disk for persistence.

## Basics

### The Model

SynDisco can theoretically support any LLM, as long as it is wrapped in a `BaseModel` wrapper. The `BaseModel` class is a very simple interface with one method. This method gives the underlying LLM input, and returns its output to the library.

There already exists a `TransformersModel` class which handles models from the `transformers` python library. In 90% of your applications, this will be enough. We can load a TransformersModel using the following code:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from syndisco.model import TransformersModel

llm = TransformersModel(
    model_path="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    name="test_model",
    max_out_tokens=100,
)

/home/dimits/miniconda3/envs/syndiscooooo/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Device set to use cuda:0


This will download a small LLM from huggingface. You can substitute the model_path for any similar model in [HuggingFace](https://huggingface.co/) supporting the Transformers library.

### Creating personas

All `actors` can be defined by a `persona`, aka a set of attributes that define them. These attributes can be age, ethnicity, and even include special instructions on how they should behave.

Creating a persona programmatically is simple:

In [3]:
from syndisco.actors import Persona

persona_data = [
    {
        "username": "Emma35",
        "age": 38,
        "sex": "female",
        "education_level": "Bachelor's",
        "sexual_orientation": "Heterosexual",
        "demographic_group": "Latino",
        "current_employment": "Registered Nurse",
        "special_instructions": "",
        "personality_characteristics": [
            "compassionate",
            "patient",
            "diligent",
            "overwhelmed",
        ],
    },
    {
        "username": "Giannis",
        "age": 21,
        "sex": "male",
        "education_level": "College",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Game Developer",
        "special_instructions": "",
        "personality_characteristics": [
            "strategic",
            "meticulous",
            "nerdy",
            "hyper-focused",
        ],
    },
]

personas = [Persona(**data) for data in persona_data]

for persona in personas:
    print(persona)

{"username": "Emma35", "age": 38, "sex": "female", "sexual_orientation": "Heterosexual", "demographic_group": "Latino", "current_employment": "Registered Nurse", "education_level": "Bachelor's", "special_instructions": "", "personality_characteristics": ["compassionate", "patient", "diligent", "overwhelmed"]}
{"username": "Giannis", "age": 21, "sex": "male", "sexual_orientation": "Pansexual", "demographic_group": "White", "current_employment": "Game Developer", "education_level": "College", "special_instructions": "", "personality_characteristics": ["strategic", "meticulous", "nerdy", "hyper-focused"]}


Since creating a lot of distinct users is essential in running large-scale experiments, users are usually defined in JSON format. That way, you can change anything without touching your code!

[Here](https://github.com/dimits-ts/synthetic_moderation_experiments/blob/master/data/discussions_input/personas/personas.json) is an applied example of how to mass-define user personas through JSON files.

### Creating the user-agents

Having a `persona` and a `model` we can finally create an `actor`. The actor will personify the selected persona using the model to talk.

Besides a persona and a model, the actors will also need instructions and a context. By convention, all actors share the same context, and all user-agents share the same instructions. Personalized instructions are defined in the actor's persona.

In [4]:
from syndisco.actors import Actor, ActorType


CONTEXT = "You are taking part in an online conversation"
INSTRUCTIONS = "Act like a human would"

actors = [
    Actor(
        model=llm,
        persona=p,
        actor_type=ActorType.USER,
        context=CONTEXT,
        instructions=INSTRUCTIONS
    )
    for p in personas
]

### Managing turn-taking

In real-life discussions, who gets to speak at each point in time is determined by complex social dynamics, which are difficult to realistically model.
However, there are ways with which we can simulate these dynamics. 

SynDisco uses the `TurnManager` class to model turn taking. Two implementations are available by default: Round Robin, and Random Weighted

- `Round Robin` is the simplest, most intuitive way to model turn-taking; everyone gets to talk once per round. Once everyone talks once, they get to talk again in the same sequence

In [5]:
from syndisco.turn_manager import RoundRobin


rrobin_turn_manager = RoundRobin(["John", "Mary", "Howard", "Todd"])

for i in range(10):
    print(next(rrobin_turn_manager))

John
Mary
Howard
Todd
John
Mary
Howard
Todd
John
Mary


- `RandomWeighted` on the other hand throws a weighted coin on each round. If the coin flip succedes, the previous user gets to respond. If not, another user is selected at random

In [6]:
from syndisco.turn_manager import RandomWeighted


rweighted_turn_manager = RandomWeighted(
    names=["John", "Mary", "Howard", "Todd"], p_respond=0.5
)

for i in range(10):
    print(next(rweighted_turn_manager))

Todd
Mary
Todd
Howard
Todd
John
Todd
Howard
Todd
John


## Generating a discussion

Let's start with the most basic task; a single discussion between two user-agents.

Since we only have two users, a RoundRobin approach where each user takes a turn sequentially is sufficient.

Now we can run a simple discussion

In [7]:
from syndisco.jobs import Discussion


turn_manager = RoundRobin([actor.get_name() for actor in actors])
conv = Discussion(next_turn_manager=turn_manager, users=actors)
conv.begin()

User Emma35 posted:
I'd love to chat with you. What's been going on in your world? 

User Giannis posted:
Not much, Emma. Just been keeping busy with work and trying to squeeze
in some gaming sessions on the side. I've been working on a new game
project and it's been a challenge, but I'm excited to see it come
together. How about you, what's new with you? 

User Emma35 posted:
I'm doing alright, I guess. Work has been keeping me pretty busy, as
usual. I've been working on a new project at the hospital and it's
been a lot of stress, but it's also been really rewarding. We're
trying to implement some new procedures to improve patient care and
it's been a challenge, but it's worth it in the end. I just wish I had
more time to focus on my own projects outside of work. I've been
wanting to start a community 

User Giannis posted:
That sounds like a really rewarding project, Emma. I can understand
why you'd want to prioritize it. As a game developer, I can relate to
the feeling of wanting to

Let's add a moderator to oversee the discussion.

In [ ]:
MODERATOR_INSTRUCTIONS = "You are a moderator. Oversee the discussion"

moderator_persona = Persona(
    **{
        "username": "Moderator",
        "age": 41,
        "sex": "male",
        "education_level": "PhD",
        "sexual_orientation": "Pansexual",
        "demographic_group": "White",
        "current_employment": "Moderator",
        "special_instructions": "",
        "personality_characteristics": [
            "strict",
            "neutral",
            "just",
        ],
    }
)

moderator = Actor(
    model=llm,
    persona=moderator_persona,
    actor_type=ActorType.USER,
    context=CONTEXT,
    instructions=MODERATOR_INSTRUCTIONS
)

# remember to update this!
turn_manager = RoundRobin(
    [actor.get_name() for actor in actors] + [moderator.get_name()]
)
conv = Discussion(
    next_turn_manager=turn_manager,
    users=actors + [moderator],
    moderator=moderator,
)
conv.begin()

User Emma35 posted:
It seems like you haven't posted anything yet, Emma35! What's on your
mind? 

User Moderator posted:
User Emma35, it's great to see you're kicking off the conversation.
I'm happy to chat with you. Since I'm the Moderator, I'll keep the
discussion on track and ensure it remains respectful and on topic.
What's been on your mind lately? 

User Giannis posted:
I've been super excited about a new project I'm working on at my job
as a game developer. We're trying to create a VR experience that
simulates a futuristic city, and I'm in charge of designing the
architecture. It's a lot of fun, but also a lot of pressure - I want
to make sure everything is just right. I've been spending hours
researching different urban planning models and experimenting with
different textures and lighting effects. Do you have any experience
with VR development or urban planning 

User Moderator posted:
User Giannis, that sounds like an exciting project. As a Moderator,
I'd love to hear more ab

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


User Moderator posted:
User Giannis, that's great to hear that you're working on a VR
experience.   To get started, can you tell us more about the
futuristic city you're envisioning? What kind of architecture,
transportation systems, and features are you planning to include, and
how do you see them fitting together to create an immersive experience
for users?  Also, have you considered any specific themes or elements
that you want to incorporate into the city, such as sustainability,
technology, or social commentary? And what 

User Moderator posted:
User Giannis, that's a great question. I envision the city to be a
sustainable and technologically advanced metropolis with a mix of
futuristic and organic architecture. The city will have a series of
interconnected districts, each with its own unique character and
theme. For example, there will be a district focused on renewable
energy, a district with a strong focus on sustainable transportation,
and a district that showcases cutting-edg

### Saving a discussion to the disk

It's best practice to save the results of each discussion after it has concluded. This way, no matter what happens to the program executing the discussions, progress will be checkpointed.

The `Discussion` class provides a method for saving its logs and related metadata to a JSON file:

In [ ]:
import tempfile
import json

tp = tempfile.NamedTemporaryFile(delete=True)

conv.to_json_file(tp.name)

# if you are running this on Windows, uncomment this line
# tp.close()
with open(tp.name, mode="rb") as f:
    print(json.dumps(json.load(f), indent=2))

Congratulations, you can now run fully synthetic discussions! You may want to experiment with adding more than 2 users or testing more realistic turn taking procedures (for example, check out the `RandomWeighted` turn manager).